In [1]:
msg = "Hello"
print(msg)

Hello


In [2]:
pip install xgboost

   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/72.0 MB ? eta -:--:--
    --------------------------------------- 1.0/72.0 MB 3.6 MB/s eta 0:00:20
    --------------------------------------- 1.6/72.0 MB 3.8 MB/s eta 0:00:19
   - -------------------------------------- 2.6/72.0 MB 3.5 MB/s eta 0:00:20
   - -------------------------------------- 3.4/72.0 MB 3.6 MB/s eta 0:00:20
   -- ------------------------------------- 3.9/72.0 MB 3.5 MB/s eta 0:00:20
   -- ------------------------------------- 5.0/72.0 MB 3.6 MB/s eta 0:00:19
   --- ------------------------------------ 5.8/72.0 MB 3.7 MB/s eta 0:00:18
   --- ------------------------------------ 6.6/72.0 MB 3.7 MB/s eta 0:00:18
   ---- ----------------------------------- 7.3/72.0 MB 3.7 MB/s eta 0:00:18
   ---- ----------------------------------- 8.1/72.0 MB 3.7 MB/s eta 0:00:18
   ---- ----------------------------------- 8.9/72.0 MB 3.7 MB/s eta 0:00:17
   ----- ----


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import xgboost as xgb
from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [8]:
# Tai du lieu gia vang tu Yahoo Finance
import yfinance as yf
data = yf.download("GC=F", start="2010-01-01", end="2020-12-31")
data.to_csv("gold_prices_xg.csv")

# Fix multi-index
data.columns = [col[0] for col in data.columns]

data.head()

C:\Users\minam\AppData\Local\Temp\ipykernel_5888\1995452583.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download("GC=F", start="2010-01-01", end="2020-12-31")
[*********************100%***********************]  1 of 1 completed


,Close,High,Low,Open,Volume
Date,,,,,
2010-01-04,1117.699951,1122.300049,1097.099976,1117.699951,184
2010-01-05,1118.099976,1126.500000,1115.000000,1118.099976,53
2010-01-06,1135.900024,1139.199951,1120.699951,1135.900024,363
2010-01-07,1133.099976,1133.099976,1129.199951,1133.099976,56
2010-01-08,1138.199951,1138.199951,1122.699951,1138.199951,54


In [12]:
# Chuẩn bị dữ liệu
X = data.drop("Close", axis=1)
y = data["Close"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)


In [13]:
#Khoi tao mo hinh
model = XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='reg:squarederror'
)

In [14]:
#Train mo hinh
model.fit(X_train, y_train)


,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.8
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [15]:
#Du doan
y_pred = model.predict(X_test)

In [ ]:
#Danh gia mo hinh
import numpy as np
import sklearn
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

print("sklearn version:", sklearn.__version__)

# Chuyển về numpy 1D
y_test_arr = np.array(y_test).ravel()
y_pred_arr = np.array(y_pred).ravel()

# In shape để debug
print("shapes:", y_test_arr.shape, y_pred_arr.shape)

# Nếu shape khác → báo lỗi rõ ràng
if y_test_arr.shape != y_pred_arr.shape:
    raise ValueError(f"Shape mismatch: y_test {y_test_arr.shape}, y_pred {y_pred_arr.shape}.")

# Tính metric
mae = mean_absolute_error(y_test_arr, y_pred_arr)
mse = mean_squared_error(y_test_arr, y_pred_arr)   # không dùng squared
rmse = np.sqrt(mse)
r2 = r2_score(y_test_arr, y_pred_arr)

print(f"MAE:  {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R2:   {r2:.4f}")

#Nhan xet: Mo hinh XGBoost cho ket qua du doan kha tot voi MAE, RMSE thap va R2 gan 1


sklearn version: 1.7.2
shapes: (553,) (553,)
MAE:  14.2779
RMSE: 29.9538
R2:   0.9839
